In [ ]:
### Fermentation Dynamic Model

In [1]:
import numpy as np
import pandas as pd
import math as m
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
%matplotlib

Using matplotlib backend: Qt5Agg


In [46]:
#layout = widgets.Layout(width='auto', height='40px')        #set width and height


# Set up widgets of all molecular weight
Mwx_widgets = widgets.FloatText(
        value=24.6,
        description = "Molweight of Biomass",
        disabled= True,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
Mwo2_widgets = widgets.FloatText(
        value=32,
        description = "Molweight of O2",
        disabled= True,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
Mwco2_widgets = widgets.FloatText(
        value=44,
        description = "Molweight of CO2",
        disabled= True,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
Mwglc_widgets = widgets.FloatText(
        value=180,
        description = "Molweight of Glucose",
        disabled= True,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
Mwnh3_widgets = widgets.FloatText(
        value=17,
        description = "Molweight of NH3",
        disabled= True,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
Mwp_widgets = widgets.FloatText(
        value=24,
        description = "Molweight of Protein",
        disabled= True,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})

# Set up widgets of fixed constant
# yO2_air = 0.21   # O2 fraction ingas (mol/mol)
# yCO2_air =0.0004    #CO2 fraction ingas (mol/mol)
# mv = 1.01*10000/8.314/(273+T)    # Mol per gas volume (mol/m3)


# Set up widgets of kinetics model coefficient
q1max_widgets = widgets.FloatText(
        value=0.1,
        description = "Maximum specific growth rate (1/h)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
K1_widgets = widgets.FloatText(
        value=0.1,
        description = "Affinity growth of glc (g/kg)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
K2_widgets = widgets.FloatText(
        value=0.1,
        description = "Affinity growth of nh3 (g/kg)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
K3_widgets = widgets.FloatText(
        value=0.1,
        description = "Ratio protein formation rate to growth (-)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
K4_widgets = widgets.FloatText(
        value=2,
        description = "Decline of protein formation towards max growth (-)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
K5_widgets = widgets.FloatText(
        value=-100,
        description = "Onset decline of protein formation rate (kg/g)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
K6_widgets = widgets.FloatText(
        value=0.1,
        description = "Onset decline of protein formation rate (g/kg)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
q4max_widgets = widgets.FloatText(
        value=4.5,
        description = "Maximum maintenance (kJ/Cmol X/h)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
K7_widgets = widgets.FloatText(
        value=0.01,
        description = "Affinity maintenance for glc (g/kg)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
G1_widgets = widgets.FloatText(
        value=240,
        description = "Energy consumption for growth (kJ/Cmol X)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
G2_widgets = widgets.FloatText(
        value=220,
        description = "Energy consumption for protein formation (kJ/Cmol P)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
G3_widgets = widgets.FloatText(
        value=2843,
        description = "Energy generation by glc catabolism (kJ/mol glc)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})


# Set up widgets of Run Initial Condition
Cglc0_widgets = widgets.FloatText(
        value=50,
        description = "Initial glucose concentration (g/kg)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
Cnh30_widgets = widgets.FloatText(
        value=3,
        description = "Initial MH3 concentration (g/kg)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
Cx0_widgets = widgets.FloatText(
        value=1,
        description = "Initial biomass concentration (g/kg)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
Cp0_widgets = widgets.FloatText(
        value=0,
        description = "Initial protein concentration (g/kg)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
WB0_widgets = widgets.FloatText(
        value=10,
        description = "Initial broth weight (kg)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
Mco2_widgets = widgets.FloatText(
        value=0,
        description = "Initial CO2 (mol)",
        disabled= True,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
RunStart_widgets = widgets.IntText(
        value=0,
        description = "Time Start (hour)",
        disabled= True,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
RunEnd_widgets = widgets.IntText(
        value=72,
        description = "Time End (Hour)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
TimeInterval_widgets = widgets.IntText(
        value=72,
        description = "Time Interval (hour)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})

# Set up widgets of Fermentation setting
# tf = 66     # Duration of fermentation (h)
# Tint = 1     # Time resolution of simulation (h)
# evap = 0     # Evaporation rate (g/h)
Evaporation_widgets = widgets.IntText(
        value=0,
        description = "Evaporation rate (g/h)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
# airflw = 10    # Airflow (Nl/min)
airflw_widgets = widgets.IntText(
        value=10,
        description = "Airflow (Nl/min",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
# Fglc0 = 0    # Initial glucose feed rate (g/h)
Fglc0_widgets = widgets.IntText(
        value=0,
        description = "Initial flucose feed rate",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
# expF = 0.1   # Exponential increase of feed rate (1/h)
expF_widgets = widgets.IntText(
        value=0,
        description = "Exponential increase of feed rate (1/h)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
# flag = 0     # Flag indication batch phase (0) or fed-batch phase (1)
flag_widgets = widgets.IntText(
        value=0,
        description = "Flag indication batch phase (0) or fed-batch phase (1)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})
# T = 25      # Fermentation T (C)
T_widgets = widgets.IntText(
        value=25,
        description = "Fermentation T (C)",
        disabled= False,
        style= {'description_width': 'initial'},
        layout= {'width': '400px'})

# Set up widgets of Feed solutions
# Cglcf = 0.3     # Glucose concentration in feed solution (kg/kg)
# Cnh3t = 0.25    # NH3 concentration in titrant (kg/kg)
# # Feed dosing:
# Fglc = 0

# # NH3 feeding:
# Fnh3 = 0        # Titrant rate (g nh3/h)

In [47]:
items_MW = [Mwx_widgets, Mwo2_widgets, Mwco2_widgets, Mwglc_widgets, Mwnh3_widgets, Mwp_widgets]
items_kinetics = [q1max_widgets, K1_widgets, K2_widgets, K3_widgets, K4_widgets, K5_widgets, K6_widgets, q4max_widgets, K7_widgets, G1_widgets, G2_widgets, G3_widgets]
items_RunConditions = [Cglc0_widgets, Cnh30_widgets, Cx0_widgets, Cp0_widgets, WB0_widgets, Mco2_widgets, RunStart_widgets, RunEnd_widgets, TimeInterval_widgets]
items_FermentationSet = [Evaporation_widgets, airflw_widgets, Fglc0_widgets, expF_widgets, flag_widgets, T_widgets]
MW = widgets.VBox(items_MW)
Coe_kinetics = widgets.VBox(items_kinetics)
RunConditions = widgets.VBox(items_RunConditions)
Fermentation_Set = widgets.VBox(items_FermentationSet)

In [48]:
tab_nest = widgets.Tab()
tab_nest.children = [MW, Coe_kinetics, RunConditions, Fermentation_Set]
tab_nest.set_title(0, "Molecular Weight")
tab_nest.set_title(1, "Kinetic Coefficient")
tab_nest.set_title(2, "Run Initial Conditions")
tab_nest.set_title(3, "Fermentation Setting")
tab_nest

In [53]:
def AssignValues():
    global Mwx,Mwo2,Mwco2,Mwglc,Mwnh3,Mwp,\
            q1max,K1,K2,K3,K4,K5,K6,q4max,K7,G1,G2,G3,\
            Cglc0,Cnh30,Cx0,Cp0,WB0,Mglc0,Mnh30,Mx0,Mp0,Mco20,\
            w,ti,tf,h,Tglc0,Tnh30,To20,Tco20,Th2o0,Fglc,Fnh3,Cglcf,Cnh3t,\
            evap,airflw
    
    # Fixed constants
    Mwx = Mwx_widgets.value  # Molweight biomass (g/mol)
    Mwo2 = Mwo2_widgets.value   # Molweight O2 (g/mol)
    Mwco2 = Mwco2_widgets.value   # Molweight CO2 (g/mol)   
    Mwglc = Mwglc_widgets.value  # Molweight glucose (g/mol)
    Mwnh3 = Mwnh3_widgets.value   # Molweight NH3 (g/mol)
    Mwp = Mwp_widgets.value  # Molweight Protein (g/mol)
    # Kinetics
    q1max = q1max_widgets.value    # Maximum specific growth rate (1/h)
    K1 = K1_widgets.value    # Affinity growth of glc (g/kg)
    K2 = K2_widgets.value   # Affinity growth of nh3 (g/kg)
    K3 = K3_widgets.value   # Ratio protrin formation rate to growth (-)
    K4 = K4_widgets.value    # Decline of protein formation towards max growth (-)
    K5 = K5_widgets.value     # Onset decline of protein formation rate (kg/g)
    K6 = K6_widgets.value    # Onset decline of protein formation rate (g/kg)
    q4max = q4max_widgets.value    # Maximum maintenance (kJ/Cmol X/h)
    K7 = K7_widgets.value    # Affinity maintenance for glc (g/kg)
    G1 = G1_widgets.value     # Energy consumption for growth (kJ/Cmol X)
    G2 = G2_widgets.value     # Energy consumption for protein formation (kJ/Cmol P)
    G3 = G3_widgets.value    # Energy generation by glc catabolism (kJ/mol glc)
    # Initial Condition
    Cglc0 = Cglc0_widgets.value   # Initial glucose concentration (g/kg)
    Cnh30 = Cnh30_widgets.value    # Initial NH3 concentration (g/kg)
    Cx0 = Cx0_widgets.value     # Initial biomass concentration (g/kg)
    Cp0 = Cp0_widgets.value     # Initial protein concentration (g/kg)

    WB0 = WB0_widgets.value    # Initial broth weight (kg)
    Mglc0 = Cglc0*WB0    # Initial glucose weight (g)
    Mnh30 = Cnh30*WB0    # Initial NH3 weight (g)
    Mx0 = Cx0*WB0    # Initial biomass weight (g)
    Mp0 = Cp0*WB0    # Initial protein weight (g)
    Mco20 = Mco2_widgets.value     # Initial CO2 (mol)
    Tglc0 = Mglc0    # Totalized glucose dosed (g)
    Tnh30 = Mnh30    # Totalized NH3 dosed (g)
    To20 = 0    # Totalized O2 consumed (mol)
    Tco20 = 0    # Totalized CO2 produced (mol)
    Th2o0 = 0    # Totalized H2O evaporated (g)

    w = np.array([WB0, Mglc0, Mnh30, Mx0, Mp0, Tglc0, Tnh30, To20, Tco20, Th2o0]) 
    ti = RunStart_widgets.value 
    tf = RunEnd_widgets.value
    h = TimeInterval_widgets.value
    
    # Fermentation Setting
    evap = Evaporation_widgets.value 
    airflw = airflw_widgets.value
    Fglc0 = Fglc0_widgets.value 
    expF = expF_widgets.value
    flag = flag_widgets.value 
    T = T_widgets.value
    
    # Set up widgets of Feed solutions
    Cglcf = 0.3     # Glucose concentration in feed solution (kg/kg)
    Cnh3t = 0.25    # NH3 concentration in titrant (kg/kg)
    # Feed dosing:
    Fglc = 0

    # NH3 feeding:
    Fnh3 = 0        # Titrant rate (g nh3/h)



In [2]:
# # Fermentation setting:
# tf = 66     # Duration of fermentation (h)
# Tint = 1     # Time resolution of simulation (h)
# evap = 0     # Evaporation rate (g/h)
# airflw = 10    # Airflow (Nl/min)
# Fglc0 = 0    # Initial glucose feed rate (g/h)
# expF = 0.1   # Exponential increase of feed rate (1/h)
# flag = 0     # Flag indication batch phase (0) or fed-batch phase (1)
# T = 25      # Fermentation T (C)


# # Fixed constants
# Mwx = 26  # Molweight biomass (g/mol)
# Mwco2 = 44   # Molweight CO2 (g/mol) 
# Mwo2 = 32    # Molweight O2 (g/mol)
# Mwglc = 180  # Molweight glucose (g/mol)
# Mwnh3 = 17   # Molweight NH3 (g/mol)
# Mwp = 24  # Molweight protein (g/mol)
# yO2_air = 0.21   # O2 fraction ingas (mol/mol)
# yCO2_air =0.0004    #CO2 fraction ingas (mol/mol)
# mv = 1.01*10000/8.314/(273+T)    # Mol per gas volume (mol/m3)

# # Feed Solutions:
# Cglcf = 0.3     # Glucose concentration in feed solution (kg/kg)
# Cnh3t = 0.25    # NH3 concentration in titrant (kg/kg)

# # Feed dosing:
# Fglc = 0

# # NH3 feeding:
# Fnh3 = 0        # Titrant rate (g nh3/h)

# # offgas





In [3]:
# # Kinetics
# q1max = 0.2    # Maximum specific growth rate (1/h)
# K1 = 0.2    # Affinity growth of glc (g/kg)
# K2 = 0.1   # Affinity growth of nh3 (g/kg)
# K3 = 0.15   # Ratio protrin formation rate to growth (-)
# K4 = 1    # Decline of protein formation towards max growth (-)
# K5 = -100     # Onset decline of protein formation rate (kg/g)
# K6 = 0.1    # Onset decline of protein formation rate (g/kg)
# q4max = 4.5    # Maximum maintenance (kJ/Cmol X/h)
# K7 = 0.01    # Affinity maintenance for glc (g/kg)
# G1 = 240     # Energy consumption for growth (kJ/Cmol X)
# G2 = 220     # Energy consumption for protein formation (kJ/Cmol P)
# G3 = 2843    # Energy generation by glc catabolism (kJ/mol glc)


In [16]:
# # Initial Condition
# Cglc0 = 50   # Initial glucose concentration (g/kg)
# Cnh30 = 3    # Initial NH3 concentration (g/kg)
# Cx0 = 1     # Initial biomass concentration (g/kg)
# Cp0 = 0     # Initial protein concentration (g/kg)

# WB0 = 10    # Initial broth weight (kg)
# Mglc0 = Cglc0*WB0    # Initial glucose weight (g)
# Mnh30 = Cnh30*WB0    # Initial NH3 weight (g)
# Mx0 = Cx0*WB0    # Initial biomass weight (g)
# Mp0 = Cp0*WB0    # Initial protein weight (g)


# Tglc0 = Mglc0    # Totalized glucose dosed (g)
# Tnh30 = Mnh30    # Totalized NH3 dosed (g)
# To20 = 0    # Totalized O2 consumed (mol)
# Tco20 = 0    # Totalized CO2 produced (mol)
# Th2o0 = 0    # Totalized H2O evaporated (g)

# w = np.array([WB0, Mglc0, Mnh30, Mx0, Mp0, Tglc0, Tnh30, To20, Tco20, Th2o0])

In [40]:
# Reaction rates
# q1 = q1max * Cglc/(K1+Cglc) * Cnh3/(K2+Cnh3)    # Specific growth rate (Cmol X/Cmol/h)
# q2 = K3 * q1 * (1-(K4/(1+m.exp(K5*(q1-K6)))))   # Specific protein formation rate (mol P/Cmol X/h)
# q4 = q4max * Cglc/(K7+Cglc)                     # Maintenance (KJ/Cmol x/h)
# q3 = (G1*q1 + G2*q2 + q4)/G3                    # Glucose catabolism (mol glc/CmolX/h)

# (1) Growth reaction
# 0.175 C6H12O6 + 0.2 NH3 + 240 KJ -> 1 C1H1.8O0.5N0.2 + 0.05 CO2 + 0.45 H2O
#
# (2) Protein formation
# 0.171 C6H12O6 + 0.3 NH3 + 220 KJ -> 1 C1H1.6O0.3N0.3 + 0.025 CO2 + 0.675 H2O
#
# (3) Glucose catabolism
# 1 C6H12O6 + 6 O2 -> 6 CO2 + 6 H2O + 2843 KJ

# qglc = 0.175*q1 + 0.1708*q2 + q3
# qnh3 = 0.2*q1 + 0.3*q2
# qO2 = 6*q3
# qco2 = 0.05*q1 + 0.025*q2 + 6*q3
# qx = q1
# qp = q2

# # State Variables
# Cglc = Mglc/WB    # Glucose concentration in broth (g/kg)
# Cnh3 = Mnh3/WB    # NH3 concentration in broth (g/kg)
# Cx = Mx/WB        # Biomass concentration in broth (g/kg)
# Cp = Mp/WB        # Protein concentration in broth (g/kg)



def function():
    global Cglc, Cnh3, Cx, Cp, q4, q1, q2, q3, qglc, qnh3, qo2, qco2, qx, qp
    Cglc = Mglc/WB    # Glucose concentration in broth (g/kg)
    Cnh3 = Mnh3/WB    # NH3 concentration in broth (g/kg)
    Cx = Mx/WB        # Biomass concentration in broth (g/kg)
    Cp = Mp/WB        # Protein concentration in broth (g/kg)
    q1 = q1max * Cglc/(K1+Cglc) * Cnh3/(K2+Cnh3)    # Specific growth rate (Cmol X/Cmol/h)
    q2 = K3 * q1 * (1-(K4/(1+m.exp(K5*(q1-K6)))))   # Specific protein formation rate (mol P/Cmol X/h)
    q4 = q4max * Cglc/(K7+Cglc)                     # Maintenance (KJ/Cmol x/h)
    q3 = (G1*q1 + G2*q2 + q4)/G3                    # Glucose catabolism (mol glc/CmolX/h)
    qglc = (0.175*q1 + 0.1708*q2 + q3)*Mwglc/Mwx    # Specific glucose consumption (g glc/ g X/h)
    qnh3 = (0.2*q1 + 0.3*q2)*Mwnh3/Mwx             # Specific NH3 consumption (g nh3/g X/h)
    qo2 =  6*q3/Mwx                                  # Specific O2 consumption (mol O2/g X/h)
    qco2 = (0.05*q1 + 0.025*q2 + 6*q3)/Mwx          # Specific CO2 consumption (mol CO2/g X/h)
    qx = q1                                         # Specific biomass growth (g X/g X/h)
    qp = q2*Mwp/Mwx                                 # Specific protein formation (g P/g X/h)
    

In [41]:
# Mass balance differenctial equation
# E1 = Change in broth weight (kg broth/h)
# E2 = Change in glucose (g glucose/h)
# E3 = Change in NH3 (g NH3/h)
# E4 = Change in biomass (g x/h)
# E5 = Change in protein (g protein/h)
# E6 = Change in totalized glucose feed (g glucose/h)
# E7 = Change in totalized NH3 feed (kg NH3/h)
# E8 = Change in totalized O2 (mol O2/h)
# E9 = Change in totalized CO2 (mol CO2/h)
# E10 = Change in totalized evaporated water (g H2O/h)


def Diffeq(t,w):                                                          
    global Mx, Mglc, Mnh3, Mp, WB, Tglc, Tnh3, To2, Tco2, Th2o
    WB, Mglc, Mnh3, Mx, Mp, Tglc, Tnh3, To2, Tco2, Th2o = w
    function()

    E1 = (Fglc/Cglcf+Fnh3/Cnh3t+qo2*Mx*Mwo2-qco2*Mx*Mwco2-evap)/1000
    E2 = Fglc - qglc*Mx
    E3 = Fnh3 - qnh3*Mx
    E4 = qx*Mx
    E5 = qp*Mx
    E6 = Fglc
    E7 = Fnh3
    E8 = qo2*Mx
    E9 = qco2*Mx
    E10 = evap
    
    return np.array([E1,E2,E3,E4,E5,E6,E7,E8,E9,E10])

In [42]:
def Kineq(Diffeq,t0,tf,h):                                                      
    w = np.array([WB0, Mglc0, Mnh30, Mx0, Mp0, Tglc0, Tnh30, To20, Tco20, Th2o0]) 
    global Mx, Mglc, Mnh3, Mp, WB, Tglc, Tnh3, To2, Tco2, Th2o
    WB, Mglc, Mnh3, Mx, Mp, Tglc, Tnh3, To2, Tco2, Th2o = w
    function()
    Y = np.empty(shape=(tf+1,10)) 
    Y1 = np.empty(shape=(tf+1,14))
    T = []                                                             
    n = (tf-t0)/h                                                         
    t = t0
    
    
    while t <= tf:
        WB, Mglc, Mnh3, Mx, Mp, Tglc, Tnh3, To2, Tco2, Th2o = w
        Y[int(t)] = w
        k1 = Diffeq(t,w)       
        w = w + k1
        Y1[int(t)] = [Cglc, Cnh3, Cx, Cp, q4, q1, q2, q3, qglc, qnh3, qo2, qco2, qx, qp]
        for i in range(len(w)):
            if w[i] <= 0:
                w[i] = 0
        T.append(t)
        t = t + n
    return Y, Y1

In [55]:
# Set up Botton to register function "Kineq"
def simulation():
    AssignValues()
    simulation.data = Kineq(Diffeq,ti,tf,h)
    column_names0 = ['WB', 'Mglc', 'Mnh3', 'Mx', 'Mp', 'Tglc', 'Tnh3', 'To2', 'Tco2', 'Th2o']
    data_df0 = pd.DataFrame(simulation.data[0], columns=column_names0)
    column_names1 = ['Cglc', 'Cnh3', 'Cx', 'Cp', 'q4', 'q1', 'q2', 'q3', 'qglc', 'qnh3', 'qo2', 'qco2', 'qx', 'qp']
    data_df1 = pd.DataFrame(simulation.data[1], columns=column_names1)
    Dataframe = pd.concat([data_df0, data_df1], axis=1)
    plot = Dataframe.plot(
    kind = "line",subplots = True, grid = True, title = "Sample Data",
    layout = (6,4), sharex = True, sharey = False, legend = True)
    
    return plot

interact_manual(simulation);

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

In [29]:
data_array = Kineq(Diffeq,0,66,66)

NameError: name 'Fglc' is not defined

In [9]:
column_names0 = ['WB', 'Mglc', 'Mnh3', 'Mx', 'Mp', 'Tglc', 'Tnh3', 'To2', 'Tco2', 'Th2o']
data_df0 = pd.DataFrame(data_array[0], columns=column_names0)
column_names1 = ['Cglc', 'Cnh3', 'Cx', 'Cp', 'q4', 'q1', 'q2', 'q3', 'qglc', 'qnh3', 'qo2', 'qco2', 'qx', 'qp']
data_df1 = pd.DataFrame(data_array[1], columns=column_names1)

In [10]:
Dataframe = pd.concat([data_df0, data_df1], axis=1)

In [11]:
pd.set_option('display.max_rows', None)
Dataframe

,WB,Mglc,Mnh3,Mx,Mp,Tglc,Tnh3,To2,Tco2,Th2o,...,q4,q1,q2,q3,qglc,qnh3,qo2,qco2,qx,qp
0,10.000000,500.000000,30.000000,10.000000,0.000000,500.0,30.0,0.000000,0.000000,0.0,...,4.499100,0.192777,0.000003,0.017857,0.357183,0.025210,0.004121,0.004491,0.192777,0.000002
1,9.999342,496.428173,29.747901,11.927773,0.000025,500.0,30.0,0.041207,0.044915,0.0,...,4.499094,0.192720,0.000003,0.017852,0.357079,0.025202,0.004120,0.004490,0.192720,0.000003
2,9.998558,492.169015,29.447294,14.226488,0.000055,500.0,30.0,0.090345,0.098473,0.0,...,4.499086,0.192649,0.000003,0.017846,0.356953,0.025193,0.004118,0.004489,0.192649,0.000003
3,9.997623,487.090826,29.088884,16.967212,0.000091,500.0,30.0,0.148934,0.162332,0.0,...,4.499077,0.192564,0.000003,0.017839,0.356800,0.025182,0.004117,0.004487,0.192564,0.000003
4,9.996509,481.036927,28.661615,20.234488,0.000134,500.0,30.0,0.218781,0.238463,0.0,...,4.499065,0.192460,0.000003,0.017830,0.356612,0.025168,0.004115,0.004485,0.192460,0.000003
5,9.995180,473.821059,28.152347,24.128810,0.000186,500.0,30.0,0.302037,0.329208,0.0,...,4.499051,0.192331,0.000003,0.017819,0.356382,0.025152,0.004112,0.004482,0.192331,0.000003
6,9.993597,465.221992,27.545470,28.769534,0.000249,500.0,30.0,0.401256,0.437351,0.0,...,4.499034,0.192172,0.000003,0.017805,0.356096,0.025131,0.004109,0.004479,0.192172,0.000003
7,9.991710,454.977265,26.822468,34.298243,0.000325,500.0,30.0,0.519469,0.566196,0.0,...,4.499012,0.191974,0.000003,0.017789,0.355740,0.025105,0.004105,0.004474,0.191974,0.000003
8,9.989464,442.775993,25.961415,40.882618,0.000417,500.0,30.0,0.660266,0.719657,0.0,...,4.498985,0.191724,0.000003,0.017768,0.355292,0.025072,0.004100,0.004469,0.191724,0.000003
9,9.986789,428.250724,24.936396,48.720815,0.000530,500.0,30.0,0.827895,0.902358,0.0,...,4.498951,0.191406,0.000003,0.017741,0.354720,0.025031,0.004094,0.004462,0.191406,0.000003


In [12]:
Dataframe.to_excel('output1.xlsx', engine='xlsxwriter')

In [12]:
fig = plt.figure()
axes = fig.subplots(nrows=2, ncols=3, sharex = True) 
Dataframe.iloc[:,0].plot(ax = axes[0,0], title = "Broth Weight")
Dataframe.iloc[:,1].plot(ax = axes[0,1])
Dataframe.iloc[:,2].plot(ax = axes[0,2])
Dataframe.iloc[:,3].plot(ax = axes[1,0])
Dataframe.iloc[:,4].plot(ax = axes[1,1])

<AxesSubplot:>

In [13]:
Dataframe.plot(subplots = True)

array([<AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>,
       <AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>,
       <AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>,
       <AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>,
       <AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>,
       <AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>],
      dtype=object)

In [67]:
a = np.array([-1,-5,-3]) 

In [68]:
for i in range(len(a)):
    if a[i] <= 0:
        a[i] = 0

In [69]:
a

array([0, 0, 0])